In [1]:
import numpy as np

## uploading the dataset

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\persi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
file = open('input.txt','rt')
text = file.read()
corpus = []

sentences = nltk.sent_tokenize(text)
for sentence in sentences:
    words = nltk.word_tokenize(sentence)
    words = [word.lower() for word in words if word.isalpha()]
    corpus.append(words)

sentence tokenization example:

In [203]:
k = np.random.choice(len(sentences))
print(f'Source sentence:\n{sentences[k]}\n\nResult sequence :\n{corpus[k]}')

Source sentence:
Once more, on pain of death, all men depart.

Result sequence :
['once', 'more', 'on', 'pain', 'of', 'death', 'all', 'men', 'depart']


## building model

In [153]:
from collections import defaultdict#Counter
from itertools import islice

In [141]:
def get_windows(seq, n):
    it = iter(seq)
    result = list(islice(it, n))
    if len(result) == n:
        # using sort for context words because the tuples of the same words have to be equivalent
        output = tuple(sorted(result[:n-1])), result[-1]
        yield output
    for elem in it:
        result = result[1:] + [elem,]
        output = tuple(sorted(result[:n-1])), result[-1]
        yield output

In [345]:
class LanguageModel:
    def __init__(self, n_gram=2, mode='word'):
        self.n_gram = n_gram
        self.mode = mode

    def build_model(self, text):
        self.model = defaultdict(lambda: defaultdict(lambda: 0))

        for sentence in text:
            for condition, word in list(get_windows(sentence, self.n_gram)):
                self.model[condition][word] += 1

        for condition in self.model:
            total_count = float(sum(self.model[condition].values()))
            for word in self.model[condition]:
                self.model[condition][word] /= total_count

    def calculate_proba(self, sentence, print_ngram_pr=True):
        words = nltk.word_tokenize(sentence)
        words = [word.lower() for word in words if word.isalpha()]
        if self.mode == 'character':
            words = list(' '.join(words))
            
        proba = 1
        for condition, word in list(get_windows(words, self.n_gram)):
            ngram_pr = self.model[condition][word]
            if print_ngram_pr:
                print(f'n-gram {*condition, word} probability = {ngram_pr:.5f}, sentence probability = {proba:.5f}')
            proba *= ngram_pr
        print(f'\nsentence probability = {proba:.5f}')

In [346]:
lm = LanguageModel(n_gram=3)
lm.build_model(corpus)
lm.calculate_proba('Our business is not unknown to the king')

n-gram ('business', 'our', 'is') probability = 0.50000, sentence probability = 1.00000
n-gram ('business', 'is', 'not') probability = 0.33333, sentence probability = 0.50000
n-gram ('is', 'not', 'unknown') probability = 0.00893, sentence probability = 0.16667
n-gram ('not', 'unknown', 'to') probability = 1.00000, sentence probability = 0.00149
n-gram ('to', 'unknown', 'the') probability = 0.50000, sentence probability = 0.00149
n-gram ('the', 'to', 'king') probability = 0.03886, sentence probability = 0.00074

sentence probability = 0.00003


In [347]:
lm.calculate_proba('Our business is not unknown to the boss')

n-gram ('business', 'our', 'is') probability = 0.50000, sentence probability = 1.00000
n-gram ('business', 'is', 'not') probability = 0.33333, sentence probability = 0.50000
n-gram ('is', 'not', 'unknown') probability = 0.00893, sentence probability = 0.16667
n-gram ('not', 'unknown', 'to') probability = 1.00000, sentence probability = 0.00149
n-gram ('to', 'unknown', 'the') probability = 0.50000, sentence probability = 0.00149
n-gram ('the', 'to', 'boss') probability = 0.00000, sentence probability = 0.00074

sentence probability = 0.00000


Let's see how often the word "boss" appeared in the text

In [440]:
for sentence in sentences:
    if 'boss' in sentence:
        print(sentence)

Lord:
Huntsman, I charge thee, tender well my hounds:
Brach Merriman, the poor cur is emboss'd;
And couple Clowder with the deep--mouth'd brach.
GREMIO:
First, as you know, my house within the city
Is richly furnished with plate and gold;
Basins and ewers to lave her dainty hands;
My hangings all of Tyrian tapestry;
In ivory coffers I have stuff'd my crowns;
In cypress chests my arras counterpoints,
Costly apparel, tents, and canopies,
Fine linen, Turkey cushions boss'd with pearl,
Valance of Venice gold in needlework,
Pewter and brass and all things that belong
To house or housekeeping: then, at my farm
I have a hundred milch-kine to the pail,
Sixscore fat oxen standing in my stalls,
And all things answerable to this portion.


The word 'boss' appears in the text once and makes sense different from the expected

# Let's left most frequent words and add 'UNK', 'START', 'END' tokens

In [399]:
class LanguageModel2:
    def __init__(self, n_gram=2, count_threshold=3):
        self.n_gram = n_gram
        self.count_threshold = count_threshold

    def build_model(self, text):
        self.model = defaultdict(lambda: defaultdict(lambda: 0))
        self.unk_token = "<UNKNOWN>"
        self.start_token = "<START>"
        self.end_token = "<END>"
        self.counts = Counter({self.start_token: len(text), self.end_token: len(text), self.unk_token:self.count_threshold+1})
        
        # compute counts
        for step, sentence in enumerate(text):
            if not step % 1000:
                print("working on {}kth line".format(step // 1000), end='\r')
            for token in sentence:
                self.counts[token] += 1
                         
        # prune vocab
        print('number of token before prunning:', len(self.counts))
        tokens_to_delete = []
        for token, count in self.counts.items():
            if count < self.count_threshold:
                self.counts["<UNKNOWN>"] += count
                tokens_to_delete.append(token)
        self.counts["<UNKNOWN>"] -= self.count_threshold
        for token in tokens_to_delete:
            self.counts.pop(token)
        self.vocab_size = len(self.counts)
        print('after prunning:', self.vocab_size)
        
        # build new dataset                     
        new_text = []
        for sentence in text:
            new_sentence = [self.start_token]
            new_sentence += [token if token in self.counts else self.unk_token for token in sentence]
            new_sentence += [self.end_token]
            new_text.append(new_sentence)
        
        # build the model
        for sentence in new_text:
            for condition, word in list(get_windows(sentence, self.n_gram)):
                self.model[condition][word] += 1

        for condition in self.model:
            total_count = float(sum(self.model[condition].values()))
            for word in self.model[condition]:
                self.model[condition][word] /= total_count

    def calculate_proba(self, sentence, print_ngram_pr=True):
        words = nltk.word_tokenize(sentence)
        words = [word.lower() for word in words if word.isalpha()]
        words = [word if word in self.counts else self.unk_token for word in words]
            
        proba = 1
        for condition, word in list(get_windows(words, self.n_gram)):
            ngram_pr = self.model[condition][word]
            if print_ngram_pr:
                print(f'n-gram {*condition, word} probability = {ngram_pr:.10f}, sentence probability = {proba:.10f}')
            proba *= ngram_pr
        print(f'\nsentence probability = {proba:.10f}')

In [400]:
lm2 = LanguageModel2(n_gram=3, count_threshold=3)
lm2.build_model(corpus)
lm2.calculate_proba('Our business is not unknown to the king')

number of token before prunning: 11141
after prunning: 4651
n-gram ('business', 'our', 'is') probability = 0.5000000000, sentence probability = 1.0000000000
n-gram ('business', 'is', 'not') probability = 0.3333333333, sentence probability = 0.5000000000
n-gram ('is', 'not', 'unknown') probability = 0.0087719298, sentence probability = 0.1666666667
n-gram ('not', 'unknown', 'to') probability = 1.0000000000, sentence probability = 0.0014619883
n-gram ('to', 'unknown', 'the') probability = 0.5000000000, sentence probability = 0.0014619883
n-gram ('the', 'to', 'king') probability = 0.0381679389, sentence probability = 0.0007309942

sentence probability = 0.0000279005


In [401]:
lm2.calculate_proba('Our business is not unknown to the boss')

n-gram ('business', 'our', 'is') probability = 0.5000000000, sentence probability = 1.0000000000
n-gram ('business', 'is', 'not') probability = 0.3333333333, sentence probability = 0.5000000000
n-gram ('is', 'not', 'unknown') probability = 0.0087719298, sentence probability = 0.1666666667
n-gram ('not', 'unknown', 'to') probability = 1.0000000000, sentence probability = 0.0014619883
n-gram ('to', 'unknown', 'the') probability = 0.5000000000, sentence probability = 0.0014619883
n-gram ('the', 'to', '<UNKNOWN>') probability = 0.0890585242, sentence probability = 0.0007309942

sentence probability = 0.0000651013


New model's prediction for the second sentence increased, even became more than the prediction for the first sentence.

# Let's combine unigram, bigrams and trigrams

In [423]:
def get_windows(seq, n):
    it = iter(seq)
    result = list(islice(it, n))
    if len(result) == n:
        output = tuple(result[:n-1]), result[-1]
        yield output
    for elem in it:
        result = result[1:] + [elem,]
        output = tuple(result[:n-1]), result[-1]
        yield output

In [426]:
class LanguageModel3:
    def __init__(self, lambda1=0.1, lambda2=0.3, lambda3=0.6, count_threshold=3):
        self.count_threshold = count_threshold
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.lambda3 = lambda3

    def build_model(self, text):
        self.unk_token = "<UNKNOWN>"
        self.start_token = "<START>"
        self.end_token = "<END>"
        self.uni_model = Counter({self.start_token: len(text), self.end_token: len(text), self.unk_token:self.count_threshold+1})
        self.bi_model = defaultdict(lambda: defaultdict(lambda: 0))
        self.tri_model = defaultdict(lambda: defaultdict(lambda: 0))
        
        # compute counts/uni-model's value
        for step, sentence in enumerate(text):
            if not step % 1000:
                print("working on {}kth line".format(step // 1000), end='\r')
            for token in sentence:
                self.uni_model[token] += 1
                         
        # prune vocab
        print('number of token before prunning:', len(self.uni_model))
        tokens_to_delete = []
        for token, count in self.uni_model.items():
            if count < self.count_threshold:
                self.uni_model["<UNKNOWN>"] += count
                tokens_to_delete.append(token)
        self.uni_model["<UNKNOWN>"] -= self.count_threshold
        for token in tokens_to_delete:
            self.uni_model.pop(token)
        self.vocab_size = len(self.uni_model)
        print('after prunning:', self.vocab_size)
        
        # build new dataset                     
        new_text = []
        for sentence in text:
            new_sentence = [self.start_token]
            new_sentence += [token if token in self.uni_model else self.unk_token for token in sentence]
            new_sentence += [self.end_token]
            new_text.append(new_sentence)
        
        # build models
        for sentence in new_text:
            for condition, word in list(get_windows(sentence, n=2)):
                self.bi_model[condition][word] += 1
            for condition, word in list(get_windows(sentence, n=3)):
                self.tri_model[condition][word] += 1
                self.tri_model[condition[::-1]][word] += 1
        # unigram model
        for word in self.uni_model:
            self.uni_model[word] /= self.vocab_size
        # bigram model
        for condition in self.bi_model:
            total_count = float(sum(self.bi_model[condition].values()))
            for word in self.bi_model[condition]:
                self.bi_model[condition][word] /= total_count
        # trigram model
        for condition in self.tri_model:
            total_count = float(sum(self.tri_model[condition].values()))
            for word in self.tri_model[condition]:
                self.tri_model[condition][word] /= total_count

    def calculate_proba(self, sentence, print_ngram_pr=True):
        words = nltk.word_tokenize(sentence)
        words = [word.lower() for word in words if word.isalpha()]
        words = [word if word in self.uni_model else self.unk_token for word in words]
            
        proba = 1
        for condition, word in list(get_windows(words, n=3)):
            ngram_pr = self.lambda1 * self.uni_model[word]
            ngram_pr += self.lambda2 * self.bi_model[condition[1]][word]
            ngram_pr += self.lambda3 * self.tri_model[condition][word]
            if print_ngram_pr:
                print(f'n-gram {*condition, word} probability = {ngram_pr:.10f}, sentence probability = {proba:.10f}')
            proba *= ngram_pr
        print(f'\nsentence probability = {proba:.10f}')

In [433]:
lm3 = LanguageModel3(count_threshold=3)
lm3.build_model(corpus)
lm3.calculate_proba('Our business is not unknown to the king')

number of token before prunning: 11141
after prunning: 4651
n-gram ('our', 'business', 'is') probability = 0.3494732316, sentence probability = 1.0000000000
n-gram ('business', 'is', 'not') probability = 0.2473876586, sentence probability = 0.3494732316
n-gram ('is', 'not', 'unknown') probability = 0.0055211669, sentence probability = 0.0864553645
n-gram ('not', 'unknown', 'to') probability = 0.7026660933, sentence probability = 0.0004773345
n-gram ('unknown', 'to', 'the') probability = 0.4350892281, sentence probability = 0.0003354068
n-gram ('to', 'the', 'king') probability = 0.0427459579, sentence probability = 0.0001459319

sentence probability = 0.0000062380


In [434]:
lm3.calculate_proba('Our business is not unknown to the boss')

n-gram ('our', 'business', 'is') probability = 0.3494732316, sentence probability = 1.0000000000
n-gram ('business', 'is', 'not') probability = 0.2473876586, sentence probability = 0.3494732316
n-gram ('is', 'not', 'unknown') probability = 0.0055211669, sentence probability = 0.0864553645
n-gram ('not', 'unknown', 'to') probability = 0.7026660933, sentence probability = 0.0004773345
n-gram ('unknown', 'to', 'the') probability = 0.4350892281, sentence probability = 0.0003354068
n-gram ('to', 'the', '<UNKNOWN>') probability = 0.2296982837, sentence probability = 0.0001459319

sentence probability = 0.0000335203
